## Backpropagation 

Backpropagation is a way to training neural networks, where we know the derivatives of each layer. It uses gradient descent as covered in the previous notebook. 

We'll no longer consider just a single input (a rather unrealistic assumption), but we will consider something fairly simple/restricted: all possible logic (boolean) functions in 4 dimensions.

Let's assume we have 4 inputs and a single output. We want to learn for any given input what the output should be.

Rather than work with boolean functions, we will create vector/matrix representations.

In [3]:
bfn_one(a1::Bool, a2::Bool, a3::Bool, a4::Bool) = a1 | a2 & (~a3 | a4)

bfn_one (generic function with 2 methods)

In [4]:
bfn_one(true, false, false, true)

true

In [8]:
bfn_one(false, false, false, true)

false

Okay, but let's generate inputs and outputs in number form (0s and 1s) so we can apply our previous techniques.

In [15]:
inputs = reverse.(Iterators.product(fill(0:1,4)...))[:]

16-element Array{NTuple{4,Int64},1}:
 (0, 0, 0, 0)
 (0, 0, 0, 1)
 (0, 0, 1, 0)
 (0, 0, 1, 1)
 (0, 1, 0, 0)
 (0, 1, 0, 1)
 (0, 1, 1, 0)
 (0, 1, 1, 1)
 (1, 0, 0, 0)
 (1, 0, 0, 1)
 (1, 0, 1, 0)
 (1, 0, 1, 1)
 (1, 1, 0, 0)
 (1, 1, 0, 1)
 (1, 1, 1, 0)
 (1, 1, 1, 1)

The code might be a bit magical if you aren't familiar with Julia, but let's not get sidetracked... it generates all possible inputs to our function. (Yes, of course I Googled how to do it!)

In [26]:
# map single input to Booleans
map(Bool, inputs[1])

# map all inputs to Booleans
inputs_b = map(ip -> map(Bool, ip), inputs)

# apply one example
bfn_one(map(Bool, inputs[1])...)

#apply all
outputs_b = map(ib -> bfn_one(ib...), inputs_b)

outputs = map(Int, outputs_b)

(inputs_b, outputs_b, outputs)

(NTuple{4,Bool}[(false, false, false, false), (false, false, false, true), (false, false, true, false), (false, false, true, true), (false, true, false, false), (false, true, false, true), (false, true, true, false), (false, true, true, true), (true, false, false, false), (true, false, false, true), (true, false, true, false), (true, false, true, true), (true, true, false, false), (true, true, false, true), (true, true, true, false), (true, true, true, true)], Bool[false, false, false, false, true, true, false, true, true, true, true, true, true, true, true, true], [0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1])

Actually we want floating point inputs and outputs (integer makes the zero-ish case unambiguous). Julia allows you to `convert` between types nicely:

In [32]:
inputs_f = convert(Array{NTuple{4,Float32}}, inputs)
outputs_f = convert(Array{Float32}, outputs)

inputs_f, outputs_f

(NTuple{4,Float32}[(0.0, 0.0, 0.0, 0.0), (0.0, 0.0, 0.0, 1.0), (0.0, 0.0, 1.0, 0.0), (0.0, 0.0, 1.0, 1.0), (0.0, 1.0, 0.0, 0.0), (0.0, 1.0, 0.0, 1.0), (0.0, 1.0, 1.0, 0.0), (0.0, 1.0, 1.0, 1.0), (1.0, 0.0, 0.0, 0.0), (1.0, 0.0, 0.0, 1.0), (1.0, 0.0, 1.0, 0.0), (1.0, 0.0, 1.0, 1.0), (1.0, 1.0, 0.0, 0.0), (1.0, 1.0, 0.0, 1.0), (1.0, 1.0, 1.0, 0.0), (1.0, 1.0, 1.0, 1.0)], Float32[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

Great. We can get started properly. We are going to pretend we don't know what this function is (and you can try swapping it out for something else and running the below code).